### Marine Heatwave Categorical Classification
Code to categorize marine heatwaves according to [Hobday et al., 2018](http://tos.org/oceanography/assets/docs/31-2_hobday.pdf).

- Categories are defined as local differences between the climatology and the 90th-percentile
- Multiples of this difference describe the different categoreis of MHWs

![MHW Ranking](Hobday_etal_2018.png)

In [102]:
import numpy as np
import os, time, datetime
from datetime import date
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series, concat
from marineHeatWaves import detect

### Data Prep
#### Load time series data from individual netCDF files

In [103]:
# Load data
files = ['30n120w.nc', '30n140w.nc', '40n160w.nc', '50n140w.nc'] 
f = 2 # index to files  <- choose to analyze a different time series by changing f from 0 to 3

lat = files[f][0:2]
lon = files[f][3:6]
print('Location: '+lat+'ºN,'+lon+'ºS')

fp = '/Users/hscannell/Desktop/data/SST_daily_time_series/' # <- change this to your path
data_files = Dataset(fp+'sst_'+files[f]) # reading the netCDF file and creating a dataset
    
# SST (deg-C)   
SST = np.squeeze(data_files.variables['sst'][:])

# Time (days since)
f_time = data_files.variables['time'][:] # time is given in days since 1800-01-01 00:00:00
ref = datetime.date(1800, 1, 1).toordinal()
sst_time = ref+f_time # adjust time as days since 0000-01-01 00:00:00

# Dates (yyyy-mm-dd)
sst_dates = [date.fromordinal(tt.astype(int)) for tt in sst_time]
sst_dates = np.array(sst_dates)

# Create a python series
data = DataFrame(SST, columns=['SST'])

print(data.head())

Location: 40ºN,160ºS
         SST
0  17.559999
1  18.469999
2  18.619999
3  18.719999
4  18.600000


#### Find marine heatwaves given our criteria

- To do this we need the [marineHeatWaves](https://github.com/ecjoliver/marineHeatWaves) module created by Eric Oliver. 
- We will set our climatology to be between 1988-2017. This is the period in which we define the seasonal 90th percentile threshold to define marine heatwaves over all the data.
- We are going to require that marine heatwaves persist for at least 5-days with no more than a 2-day drop in temperature below the threshold for any given event. 

In [104]:
sst = np.array(data['SST'])
time = np.array(sst_time).astype(int)

mhw, clim = detect(time, sst, climatologyPeriod=[1988, 2017], 
       pctile=90, windowHalfWidth=5, smoothPercentile=True, 
       smoothPercentileWidth=31, minDuration=6, 
       joinAcrossGaps=True, maxGap=2, maxPadLength=False, 
       coldSpells=False, alternateClimatology=False)

sst_clim = clim['seas'] # seasonal climatology
mhw_thres = clim['thresh'] # marine heatwave threshold
mhw_st = mhw['index_start'] # Start index of MHWs
mhw_en = mhw['index_end'] # End index of MHWs

#### Find where the sea surface temperature meets our criteria for a marine heatwave

(i.e. exceeds 90th percentile threshold for at least 5 days)

In [ ]:
# create MHW labels [1=MHW, 0=normal]
mhw_label = sst*0
for i in range(0,len(mhw_st)):
    n = mhw_en[i]-mhw_st[i]
    mhw_label[mhw_st[i]:mhw_en[i]] = np.ones(n)

    
# Plot binary classification
plt.rc('xtick', labelsize=16) 
plt.rc('ytick', labelsize=16)
plt.figure(figsize=(14,2))
ax = plt.subplot(111)
plt.bar(sst_dates, mhw_label, width=10, color='xkcd:reddish')
plt.yticks([0, .9], ['normal', 'marine heatwave']);
plt.title('Binary Classification', fontsize=18)
ax.get_yaxis().tick_left() 
ax.spines["right"].set_visible(False) 
ax.spines["left"].set_visible(False) 
ax.tick_params(axis=u'both', which=u'both', length=0)


print('number of marine heatwave days = \t\t{}'.format(sum(mhw_label)))
print('percent of days considered a marine heatwave = \t{}'.format(sum(mhw_label)/len(mhw_label)))

number of marine heatwave days = 		801.0
percent of days considered a marine heatwave = 	0.05962927119779647


### Create one hot encoding for marine heatwave intensity
Label days according to categorical scheme proposed in [Hobday et al., 2018](http://tos.org/oceanography/assets/docs/31-2_hobday.pdf).

* 0 = no marine heatwave
* 1 = **moderate** (1-2x threshold)
* 2 = **Strong** (2-3x threshold)
* 3 = **Severe** (3-4x threshold)
* 4 = **Extreme** (>4x threshold)

In [ ]:
M = (mhw_label == 1) # index to where there are labeled marine heatwaves
print('number of marine heatwave days = \t\t{}'.format(sum(M)))

# Find how many times the anomaly is divisable by the threshold different from the climatology
mhw_relThreshNorm = (data['SST'][M] - clim['thresh'][M])/ (clim['thresh'][M] - clim['seas'][M]) 
categs = np.zeros(data['SST'].size) 
categs[M] = np.floor(1. + mhw_relThreshNorm) 

# Plot one hot encoding for marine heatwave intensity
plt.figure(figsize=(14,4))        
plt.rc('xtick', labelsize=16) 
plt.rc('ytick', labelsize=16)
ax = plt.subplot(111)
plt.bar(sst_dates[(categs==1)], categs[(categs==1)], width=100, color='xkcd:marigold')
plt.bar(sst_dates[(categs==2)], categs[(categs==2)], width=100, color='xkcd:bright orange')
plt.bar(sst_dates[(categs==3)], categs[(categs==3)], width=100, color='xkcd:lipstick red')
plt.bar(sst_dates[(categs==4)], categs[(categs==4)], width=100, color='xkcd:reddy brown')
plt.title('MHW Categories - one hot ', fontsize=18)
plt.yticks([0, 1, 2, 3, 4], ['', 'moderate','strong','severe','extreme']);
plt.grid(True,alpha = 0.3)
ax.get_xaxis().tick_bottom()  
ax.get_yaxis().tick_left() 
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False) 
ax.spines["left"].set_visible(False) 
ax.tick_params(axis=u'both', which=u'both', length=0)

#### Export one hot encodings to a CSV

In [ ]:
import csv

fn = 'mhw_onehot_'+files[f][:-3]+'.csv'
download_dir = '/Users/hscannell/Desktop/MHWpredict/data/Labels/'+fn 
print(download_dir)
#where you want the file to be downloaded to 

MyData = np.vstack((sst_dates, categs))

myFile = open(download_dir, "w") 

with myFile:
    
    writer = csv.writer(myFile)

    writer.writerows(MyData)
     
print("Writing complete!")